<a href="https://colab.research.google.com/github/tolerant-code/Admin-Dashboard-Template-Bootstrap/blob/master/Copy_of_HT02_Support_Vector_Machine_(SVM)_Hyperparameter_Tuning_In_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources

- [Blog post](https://medium.com/grabngoinfo/support-vector-machine-svm-hyperparameter-tuning-in-python-a65586289bcb) for this notebook
- Video tutorial on [YouTube](https://www.youtube.com/watch?v=203zKEecHgg&list=PLVppujud2yJryD5u6oPjIf2LTci3dlCJG&index=1)
- More video tutorials on [hyperparameter tuning and regularization](https://www.youtube.com/playlist?list=PLVppujud2yJryD5u6oPjIf2LTci3dlCJG)
- More blog post on [hyperparameter tuning and regularization](https://medium.com/@AmyGrabNGoInfo/list/hyperparameter-tuning-2b59aca544c8)


For more information about data science and machine learning, please check out my [YouTube channel](https://www.youtube.com/channel/UCmbA7XB6Wb7bLwJw9ARPcYg), [Medium Page](https://medium.com/@AmyGrabNGoInfo) and [GrabNGoInfo.com](https://grabngoinfo.com/tutorials/).

# Intro

Support Vector Machine (SVM) is a supervised machine learning model for classifications and regressions. Since SVM is commonly used for classification, we will use the classification model as an example in this tutorial. We will cover:

* What's the intuition for the Support Vector Machine (SVM) algorithm?
* What are the most critical hyperparameters for Support Vector Machine (SVM)?
* How to tune hyperparameters for Support Vector Machine (SVM) using grid search, random search, and Bayesian optimization?

Let's get started!

# Step 1: Support Vector Machine (SVM) algorithm

In step 1, we will discuss the intuition behind the Support Vector Machine (SVM) algorithm. At a high level, the algorithm follows three steps:
1. Create hyperplanes that separate the classes.
2. Compare the margin of the hyperplanes and pick the hyperplane with the largest margin. 
 * **Margin** is the shortest distance between the hyperplane and the data points. 
 * **Maximal Margin Classifier** picks a hyperplane that maximizes the margin. One drawback of the  Maximal Margin Classifier is that it is sensitive to the outliers in the training dataset.
 * **Support Vector Machine (SVM)** solves the sensitivity problem by allowing misclassifications. The margin is called a **soft margin** when misclassifications are permitted. Therefore, a **Support Vector Classifier** is also called a **Soft Margin Classifier**. The data points on edge and within the soft margin are called **support vectors**.
 * The number of misclassifications allowed in the soft margin is determined by comparing the cross-validation results. The one with the best cross-validation result will be selected. 
3. Make predictions for the new data points based on which side of the hyperplane the new data falls in.

# Step 2: Support Vector Machine (SVM) Hyperparameters

In step 2, we will discuss the hyperparameters for Support Vector Machine (SVM). 

In python's `sklearn` implementation of the Support Vector Classification model, there is a list of different hyperparameters. You can check out the complete list in the sklearn documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). The most critical hyperparameters for SVM are `kernel`, `C`, and `gamma`.

* `kernel` function transforms the training dataset into higher dimensions to make it linearly separatable. The default kernel function for the python implementation of the support vector classifier is the Radial Basis Function, which is usually referred to as `rbf`. The kernel function can take other values such as `linear`, `poly`, `rbf`, `sigmoid`, `precomputed`, or callable.

* `C` is the l2 regularization parameter. The value of `C` is inversely proportional to the strength of the regularization. To learn more about regularization, please check out my previous tutorial on [LASSO (L1) Vs Ridge (L2) Vs Elastic Net Regularization For Classification Model](https://grabngoinfo.com/lasso-l1-vs-ridge-l2-vs-elastic-net-regularization-for-classification-model/)
  * When `C` is small, the penalty for misclassification is small, and the strength of the regularization is large. So a decision boundary with a large margin will be selected.
  * When `C` is large, the penalty for misclassification is large, and the strength of the regularization is small. A decision boundary with a small margin will be selected to reduce misclassifications.

* `gamma` is the kernel coefficient for `rbf`, `poly`, and `sigmoid`. It can be seen as the inverse of the support vector influence radius. The gamma parameter highly impacts the model performance. Gamma can take the value of `scale`, `auto`, or a float value. The default value for the python `sklearn` implementation is `scale` since version 0.22.
 * When `gamma` is small, the support vector influence radius is high. If the `gamma` value is too small, the radius of the support vectors covers the whole training dataset, and the pattern of the data will not be captured.
 * When `gamma` is large, the support vector influence radius is low. If the gamma value is too large, the support vector radius is too small to utilize `C` to prevent overfitting.

# Step 3: Import Libraries

In the third step, let's import the Python libraries needed for this tutorial.

We will use the breast cancer dataset for this tutorial, so `datasets` from `sklearn` needs to be imported. `pandas` and `numpy` are imported for data processing. `StandardScaler` is for data standardization.

For model training, we imported `train_test_split` for creating training and testing datasets and `SVC` for the support vector classification model.

For hyperparameter tuning, we imported `StratifiedKFold`, `GridSearchCV`, `RandomizedSearchCV` from sklearn. We also imported `hyperopt` and `cross_val_score` for Bayesian optimization.

In [ ]:
# Dataset
from sklearn import datasets

# Data processing
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Modeling 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
#from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

# Step 4: Read Data

In the fourth step, the breast cancer data from the `sklearn` library is loaded and transformed into a pandas dataframe.

In [ ]:
# Load the breast cancer dataset
data = datasets.load_breast_cancer()

# Put the data in pandas dataframe format
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target']=data.target

# Check the data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

The information summary shows that the dataset has 569 records and 31 columns.

In [ ]:
# Check the target value distribution
df['target'].value_counts(normalize=True)

1    0.627417
0    0.372583
Name: target, dtype: float64

The target variable distribution shows 63% of ones and 37% of zeros in the dataset. One means the patient has breast cancer, and 0 represents the patient does not have breast cancer.

# Step 5: Train Test Split

In step 5, we split the dataset into 80% training and 20% testing dataset. random_state makes the random split results reproducible.

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df[df.columns.difference(['target'])], df['target'], test_size=0.2, random_state=42)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(X_train)} records.')
print(f'The testing dataset has {len(X_test)} records.')

The training dataset has 455 records.
The testing dataset has 114 records.


The training dataset has 455 records, and the testing dataset has 114 records.

# Step 6: Standardization

In step 6, we will standardize the features.

Standardization is to change the features to the same scale. It is calculated by extracting the mean and divided by the standard deviation of each feature. After standardization, each feature has zero mean and unit standard deviation.

Standardization should be fit on the training dataset only to prevent test dataset information from leaking into the training process. Then, the test dataset is standardized using the fitting results from the training dataset.

There are different types of scalers. `StandardScaler` and `MinMaxScaler` are most commonly used. For a dataset with outliers, we can use `RobustScaler`.

In this tutorial, we will use `StandardScaler`.

In [ ]:
# Initiate scaler
sc = StandardScaler()

# Standardize the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train),index=X_train.index, columns=X_train.columns)

# Standardized the testing dataset
X_test_transformed = pd.DataFrame(sc.transform(X_test),index=X_test.index, columns=X_test.columns)

# Summary statistics after standardization
X_train_transformed.describe().T

,count,mean,std,min,25%,50%,75%,max
area error,455.0,1.561632e-17,1.001101,-0.705091,-0.464164,-0.325347,0.077435,10.641841
compactness error,455.0,-2.401498e-15,1.001101,-1.258102,-0.694353,-0.280607,0.358304,5.905671
concave points error,455.0,3.279428e-16,1.001101,-1.891775,-0.668493,-0.126279,0.437566,6.504667
concavity error,455.0,2.347329e-16,1.001101,-1.022218,-0.551340,-0.207836,0.303371,11.310294
fractal dimension error,455.0,5.050905e-16,1.001101,-1.050856,-0.573964,-0.218908,0.245340,9.345870
mean area,455.0,-2.627935e-16,1.001101,-1.365036,-0.660205,-0.289597,0.319339,5.208312
mean compactness,455.0,1.003349e-15,1.001101,-1.607228,-0.777087,-0.241340,0.528128,3.964311
mean concave points,455.0,5.780480e-16,1.001101,-1.269910,-0.734905,-0.391123,0.673757,4.022271
mean concavity,455.0,9.782163e-16,1.001101,-1.119899,-0.750539,-0.344646,0.547387,4.256736
mean fractal dimension,455.0,-3.347993e-15,1.001101,-1.776889,-0.709792,-0.177285,0.464223,4.815921


We can see that after using StandardScaler, all the features have zero mean and unit standard deviation.

Let's get the summary statistics for the training data before standardization as well, and we can see that the mean and standard deviation can be very different in scale. For example, the area error has a mean value of 40 and a standard deviation of 47. On the other hand, the compactness error has a mean of about 0.026 and a standard deviation of 0.019.

In [ ]:
# Summary statistics before standardization
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
area error,455.0,40.071299,47.236319,6.802000,18.170000,24.720000,43.725000,542.20000
compactness error,455.0,0.025635,0.018607,0.002252,0.012730,0.020420,0.032295,0.13540
concave points error,455.0,0.011894,0.006294,0.000000,0.007691,0.011100,0.014645,0.05279
concavity error,455.0,0.032824,0.032146,0.000000,0.015120,0.026150,0.042565,0.39600
fractal dimension error,455.0,0.003820,0.002787,0.000895,0.002222,0.003211,0.004504,0.02984
mean area,455.0,654.377582,354.943187,170.400000,420.300000,551.700000,767.600000,2501.00000
mean compactness,455.0,0.103619,0.052470,0.019380,0.062890,0.090970,0.131300,0.31140
mean concave points,455.0,0.048280,0.038060,0.000000,0.020340,0.033410,0.073895,0.20120
mean concavity,455.0,0.088898,0.079468,0.000000,0.029320,0.061540,0.132350,0.42680
mean fractal dimension,455.0,0.062757,0.007210,0.049960,0.057645,0.061480,0.066100,0.09744


# Step 7: Support Vector Machine (SVM) Default Hyperparameters

In step 7, we will create a Support Vector Machine (SVM) model with default hyperparameters as the baseline model.

In [ ]:
# Check default values
svc = SVC()
params = svc.get_params()
params_df = pd.DataFrame(params, index=[0])
params_df.T

,0
C,1.0
break_ties,False
cache_size,200
class_weight,None
coef0,0.0
decision_function_shape,ovr
degree,3
gamma,scale
kernel,rbf
max_iter,-1


We can see that the default hyperparameter has the C value of 1, the gamma value of `scale`, and the kernel value of `rbf`.

Next, let's fit the model using the standardized training data and check the accuracy score. We get 98.25% accuracy for the default hyperparameters.

In [ ]:
# Run model
svc.fit(X_train_transformed, y_train)

# Accuracy score
print(f'The accuracy score of the model is {svc.score(X_test_transformed, y_test):.4f}')

The accuracy score of the model is 0.9825


# Step 8: Hyperparameter Tuning Using Grid Search

In step 8, we will use grid search to find the best hyperparameter combinations for the Support Vector Machine (SVM) model. Grid search is an exhaustive hyperparameter search method. It trains models for every combination of specified hyperparameter values. Therefore, it can take a long time to run if we test out more hyperparameters and values, especially for larger datasets.

For this reason, we would like to have the grid search space relatively small so the process can finish in a reasonable timeframe. The search space includes the hyperparameters and their values grid search builds models for. In this example, we will tune three hyperparameters, C, gamma, and kernel. The other hyperparameters can be tuned in the same way.

Using the `logspace` function from the `numpy` library, we created three values for `C` and three values for `gamma`. For `gamma`, the `sklearn` values of `'scale'` and `'auto'` are also included, so there are a total of 5 values for `gamma`. 

In [ ]:
# List of C values
C_range = np.logspace(-1, 1, 3)
print(f'The list of values for C are {C_range}')

# List of gamma values
gamma_range = np.logspace(-1, 1, 3)
print(f'The list of values for gamma are {gamma_range}')

The list of values for C are [ 0.1  1.  10. ]
The list of values for gamma are [ 0.1  1.  10. ]


Two kernels, `'rbf'` and `'poly'`, will be tested.

Scoring is the metric to evaluate the cross-validation results for each model. We set `scoring = ['accuracy']`. The scoring option can take more than one metric in the list.

`StratifiedKFold` is used for the cross-validation. It helps us keep the class ratio in the folds the same as the training dataset. `n_splits=3` means we are doing 3-fold cross-validation. `shuffle=True` means the data are shuffled before splitting. `random_state=0` makes the shuffle reproducible.

In [ ]:
# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": ['rbf', 'poly'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }

# Set up score
scoring = ['accuracy']

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

We specified a few options for `GridSearchCV`.
* `estimator=svc` means we are using Support Vector Classifier as the model.
* `param_grid=param_grid` takes our pre-defined search space for the grid search.
* `scoring=scoring` set the performance evaluation metric. Because we set the scoring to 'accuracy', the model will use accuracy as the evaluation metric.
* `refit='accuracy'` enables refitting the model with the best parameters on the whole training dataset.
* `n_jobs=-1` means parallel processing using all the processors.
* `cv=kfold` takes the `StratifiedKFold` we defined.
* `verbose` controls the number of messages returned by the grid search. The higher the number, the more information is returned. `verbose=0` means silent.

In [ ]:
# Define grid search
grid_search = GridSearchCV(estimator=svc, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

# Fit grid search
grid_result = grid_search.fit(X_train_transformed, y_train)

# Print grid search summary
grid_result

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([ 0.1,  1. , 10. ]),
                         'gamma': [0.1, 1.0, 10.0, 'scale', 'auto'],
                         'kernel': ['rbf', 'poly']},
             refit='accuracy', scoring=['accuracy'])

In [ ]:
# Print the best accuracy score for the training dataset
print(f'The best accuracy score for the training dataset is {grid_result.best_score_:.4f}')

# Print the hyperparameters for the best score
print(f'The best hyperparameters are {grid_result.best_params_}')

# Print the best accuracy score for the testing dataset
print(f'The accuracy score for the testing dataset is {grid_search.score(X_test_transformed, y_test):.4f}')

The best accuracy score for the training dataset is 0.9693
The best hyperparameters are {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}
The accuracy score for the testing dataset is 0.9825


The grid search cross-validation results show that the C value of 1, gamma value of `scale` and kernel of `'rbf'` gave us the best results. This happened to be the same hyperparameters as the default value. The best average training accuracy is 96.93%, and the accuracy score for the testing dataset is 98.25%, which is the same as the baseline model.

# Step 9: Hyperparameter Tuning Using Random Search

In step 9, we use a random search for Support Vector Machine (SVM) hyperparameter tuning. Since random search randomly picks a subset of hyperparameter combinations, we can afford to try more values. 

If at least one of the parameters is a distribution, sampling with replacement is used for a random search. Sampling without replacement is used if all parameters are provided as a list. Each list is treated as a uniform distribution.

We increased the number of `C` and `gamma` values from 3 to 21 for the random search. For `gamma`, the `sklearn` values of `'scale'` and `'auto'` are also included, so there are a total of 23 values for `gamma`. 

In [ ]:
# List of C values
C_range = np.logspace(-10, 10, 21)
print(f'The list of values for C are {C_range}')

# List of gamma values
gamma_range = np.logspace(-10, 10, 21)
print(f'The list of values for gamma are {gamma_range}')

The list of values for C are [1.e-10 1.e-09 1.e-08 1.e-07 1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01
 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04 1.e+05 1.e+06 1.e+07 1.e+08 1.e+09
 1.e+10]
The list of values for gamma are [1.e-10 1.e-09 1.e-08 1.e-07 1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01
 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04 1.e+05 1.e+06 1.e+07 1.e+08 1.e+09
 1.e+10]


The same scoring metric and cross-validation values used in grid search are used for the random search. But for a random search, we need to specify a value for `n_iter`, the number of parameter combinations sampled. We are randomly testing 100 combinations for this example.

In [ ]:
# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": ['rbf', 'poly'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }

# Set up score
scoring = ['accuracy']

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Define random search
random_search = RandomizedSearchCV(estimator=svc, 
                           param_distributions=param_grid, 
                           n_iter=100,
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

# Fit grid search
random_result = random_search.fit(X_train_transformed, y_train)

# Print grid search summary
random_result

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
                   estimator=SVC(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([1.e-10, 1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03,
       1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10]),
                                        'gamma': [1e-10, 1e-09, 1e-08, 1e-07,
                                                  1e-06, 1e-05, 0.0001, 0.001,
                                                  0.01, 0.1, 1.0, 10.0, 100.0,
                                                  1000.0, 10000.0, 100000.0,
                                                  1000000.0, 10000000.0,
                                                  100000000.0, 1000000000.0,
                                                  10000000000.0, 'scale',
                                                  'auto'],
                            

The random search cross-validation results show that the C value of 1000, gamma value of 0.0001 and kernel of 'rbf' gave us the best results. The best average training accuracy is 97.14%, and the accuracy score for the testing dataset is 97.37%, which is slightly lower than the grid search results.

Note that because each time the random search is performed, a set of different hyperparameters will be randomly selected, so the best hyperparameters can be different each time random search is conducted.

In [ ]:
# Print the best accuracy score for the training dataset
print(f'The best accuracy score for the training dataset is {random_result.best_score_:.4f}')

# Print the hyperparameters for the best score
print(f'The best hyperparameters are {random_result.best_params_}')

# Print the best accuracy score for the testing dataset
print(f'The accuracy score for the testing dataset is {random_search.score(X_test_transformed, y_test):.4f}')

The best accuracy score for the training dataset is 0.9714
The best hyperparameters are {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1000.0}
The accuracy score for the testing dataset is 0.9737


# Step 10: Hyperparameter Tuning Using Bayesian Optimization

In step 10, we apply Bayesian optimization on the same search space as the random search.

There are different types of Bayesian optimization. `Hyperopt` is used in this example.

We defined an objective function that takes in the parameters and returns the loss. Since the goal is to maximize the accuracy value, we set `max(scores)` as the `best_score`, and set the loss to be `-best_score`. This setting ensures maximizing accuracy while minimizing the loss.

`fmin` is used to optimize the objective function. `Hyperopt` currently has three algorithms. They are random search, Tree of Parzen Estimators (TPE), and adaptive TPE. We are using TPE as the search algorithm.

In [ ]:
# Space
space = {
    'C' : hp.choice('C', C_range),
    'gamma' : hp.choice('gamma', gamma_range.tolist()+['scale', 'auto']),
    'kernel' : hp.choice('kernel', ['rbf', 'poly'])
}

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Objective function
def objective(params):
    
    svc = SVC(**params)
    scores = cross_val_score(svc, X_train_transformed, y_train, cv=kfold, scoring='accuracy', n_jobs=-1)

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Trials to track progress
bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = bayes_trials)

100%|██████████| 100/100 [00:06<00:00, 16.61it/s, best loss: -0.9867549668874173]


After the Bayesian optimization search, we get the best loss of -0.9868, meaning that the accuracy value is 98.68%.

We can print out the parameters for best loss and their index in the search space. We got the `C` value of 1, `gamma` value of 0.1, and `kernel` value of `'poly'`.

In [ ]:
# Print the index of the best parameters
print(best)

# Print the values of the best parameters
print(space_eval(space, best))

{'C': 10, 'gamma': 9, 'kernel': 1}
{'C': 1.0, 'gamma': 0.1, 'kernel': 'poly'}


Next, we apply the best hyperparameters to the `SVC` and make predictions. It gives an accuracy score of 94.74%. 

Theoretically, the Bayesian optimization algorithm is more efficient than the random search algorithm. This is because random search picks hyperparameters for each model independently, while Bayesian optimization utilizes previous models' information when choosing the hyperparameters for the next model.

The Bayesian optimization best model performance is not as good, probably because the number of `max_evals = 100` is not large enough for the algorithm to find the optimal values.

In [ ]:
# Train model using the best parameters
svc_bo = SVC(C=space_eval(space, best)['C'], gamma=space_eval(space, best)['gamma'], kernel=space_eval(space, best)['kernel']).fit(X_train_transformed,y_train)

# Print the best accuracy score for the testing dataset
print(f'The accuracy score for the testing dataset is {svc_bo.score(X_test_transformed, y_test):.4f}')

The accuracy score for the testing dataset is 0.9474


# Step 11: Put All Code Together

In [ ]:
###### Step 1: Support Vector Machine (SVM) algorithm

# No code in this step


###### Step 2: Support Vector Machine (SVM) Hyperparameters

# No code in this step


###### Step 3: Import Libraries

# Dataset
from sklearn import datasets

# Data processing
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Modeling 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval


###### Step 4: Read Data

# Load the breast cancer dataset
data = datasets.load_breast_cancer()

# Put the data in pandas dataframe format
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target']=data.target

# Check the data information
df.info()

# Check the target value distribution
df['target'].value_counts(normalize=True)


###### Step 5: Train Test Split

# Train test split
X_train, X_test, y_train, y_test = train_test_split(df[df.columns.difference(['target'])], df['target'], test_size=0.2, random_state=42)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(X_train)} records.')
print(f'The testing dataset has {len(X_test)} records.')


###### Step 6: Standardization

# Initiate scaler
sc = StandardScaler()

# Standardize the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train),index=X_train.index, columns=X_train.columns)

# Standardized the testing dataset
X_test_transformed = pd.DataFrame(sc.transform(X_test),index=X_test.index, columns=X_test.columns)

# Summary statistics after standardization
X_train_transformed.describe().T


###### Step 7: Support Vector Machine (SVM) Default Hyperparameters

# Check default values
svc = SVC()
params = svc.get_params()
params_df = pd.DataFrame(params, index=[0])
params_df.T

# Run model
svc.fit(X_train_transformed, y_train)

# Accuracy score
print(f'The accuracy score of the model is {svc.score(X_test_transformed, y_test):.4f}')


###### Step 8: Hyperparameter Tuning Using Grid Search

# List of C values
C_range = np.logspace(-1, 1, 3)
print(f'The list of values for C are {C_range}')

# List of gamma values
gamma_range = np.logspace(-1, 1, 3)
print(f'The list of values for gamma are {gamma_range}')

# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": ['rbf', 'poly'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }

# Set up score
scoring = ['accuracy']

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Define grid search
grid_search = GridSearchCV(estimator=svc, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

# Fit grid search
grid_result = grid_search.fit(X_train_transformed, y_train)

# Print grid search summary
grid_result

# Print the best accuracy score for the training dataset
print(f'The best accuracy score for the training dataset is {grid_result.best_score_:.4f}')

# Print the hyperparameters for the best score
print(f'The best hyperparameters are {grid_result.best_params_}')

# Print the best accuracy score for the testing dataset
print(f'The accuracy score for the testing dataset is {grid_search.score(X_test_transformed, y_test):.4f}')


###### Step 9: Hyperparameter Tuning Using Random Search

# List of C values
C_range = np.logspace(-10, 10, 21)
print(f'The list of values for C are {C_range}')

# List of gamma values
gamma_range = np.logspace(-10, 10, 21)
print(f'The list of values for gamma are {gamma_range}')

# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": ['rbf', 'poly'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": gamma_range
    }

# Set up score
scoring = ['accuracy']

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Define random search
random_search = RandomizedSearchCV(estimator=svc, 
                           param_distributions=param_grid, 
                           n_iter=100,
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

# Fit grid search
random_result = random_search.fit(X_train_transformed, y_train)

# Print grid search summary
random_result

# Print the best accuracy score for the training dataset
print(f'The best accuracy score for the training dataset is {random_result.best_score_:.4f}')

# Print the hyperparameters for the best score
print(f'The best hyperparameters are {random_result.best_params_}')

# Print the best accuracy score for the testing dataset
print(f'The accuracy score for the testing dataset is {random_search.score(X_test_transformed, y_test):.4f}')


###### Step 10: Hyperparameter Tuning Using Bayesian Optimization

# Space
space = {
    'C' : hp.choice('C', C_range),
    'gamma' : hp.choice('gamma', gamma_range),
    'kernel' : hp.choice('kernel', ['rbf', 'poly'])
}

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Objective function
def objective(params):
    
    svc = SVC(**params)
    scores = cross_val_score(svc, X_train_transformed, y_train, cv=kfold, scoring='accuracy', n_jobs=-1)

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Trials to track progress
bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = bayes_trials)

# Print the index of the best parameters
print(best)

# Print the values of the best parameters
print(space_eval(space, best))

# Train model using the best parameters
svc_bo = SVC(C=space_eval(space, best)['C'], gamma=space_eval(space, best)['gamma'], kernel=space_eval(space, best)['kernel']).fit(X_train_transformed,y_train)

# Print the best accuracy score for the testing dataset
print(f'The accuracy score for the testing dataset is {svc_bo.score(X_test_transformed, y_test):.4f}')

# Summary

In this tutorial, we covered how to tune Support Vector Machine (SVM) hyperparameters using Python. you learned:
* What's the intuition for the Support Vector Machine (SVM) algorithm?
* What are the most important hyperparameters for Support Vector Machine (SVM)?
* How to do hyperparameter tuning for Support Vector Machine (SVM) in Python using grid search, random search, and Bayesian optimization?

We used a toy dataset for this example to illustrate the hyperparameter tuning process. In real projects, I usually see random search and Bayesian optimization produce the best model performances.

# Recommended Tutorials

- [GrabNGoInfo Machine Learning Tutorials Inventory](https://medium.com/grabngoinfo/grabngoinfo-machine-learning-tutorials-inventory-9b9d78ebdd67)
- [Hierarchical Topic Model for Airbnb Reviews](https://medium.com/p/hierarchical-topic-model-for-airbnb-reviews-f772eaa30434)
- [3 Ways for Multiple Time Series Forecasting Using Prophet in Python](https://medium.com/p/3-ways-for-multiple-time-series-forecasting-using-prophet-in-python-7a0709a117f9)
- [Time Series Anomaly Detection Using Prophet in Python](https://medium.com/grabngoinfo/time-series-anomaly-detection-using-prophet-in-python-877d2b7b14b4)
- [Time Series Causal Impact Analysis in Python](https://medium.com/grabngoinfo/time-series-causal-impact-analysis-in-python-63eacb1df5cc)
- [Hyperparameter Tuning For XGBoost](https://medium.com/p/hyperparameter-tuning-for-xgboost-91449869c57e)
- [Four Oversampling And Under-Sampling Methods For Imbalanced Classification Using Python](https://medium.com/p/four-oversampling-and-under-sampling-methods-for-imbalanced-classification-using-python-7304aedf9037)
- [Five Ways To Create Tables In Databricks](https://medium.com/grabngoinfo/five-ways-to-create-tables-in-databricks-cd3847cfc3aa)
- [Explainable S-Learner Uplift Model Using Python Package CausalML](https://medium.com/grabngoinfo/explainable-s-learner-uplift-model-using-python-package-causalml-a3c2bed3497c)
- [One-Class SVM For Anomaly Detection](https://medium.com/p/one-class-svm-for-anomaly-detection-6c97fdd6d8af)
- [Recommendation System: Item-Based Collaborative Filtering](https://medium.com/grabngoinfo/recommendation-system-item-based-collaborative-filtering-f5078504996a)
- [Hyperparameter Tuning for Time Series Causal Impact Analysis in Python](https://medium.com/grabngoinfo/hyperparameter-tuning-for-time-series-causal-impact-analysis-in-python-c8f7246c4d22)
- [Hyperparameter Tuning and Regularization for Time Series Model Using Prophet in Python](https://medium.com/grabngoinfo/hyperparameter-tuning-and-regularization-for-time-series-model-using-prophet-in-python-9791370a07dc)
- [Multivariate Time Series Forecasting with Seasonality and Holiday Effect Using Prophet in Python](https://medium.com/p/multivariate-time-series-forecasting-with-seasonality-and-holiday-effect-using-prophet-in-python-d5d4150eeb57)
- [LASSO (L1) Vs Ridge (L2) Vs Elastic Net Regularization For Classification Model](https://medium.com/towards-artificial-intelligence/lasso-l1-vs-ridge-l2-vs-elastic-net-regularization-for-classification-model-409c3d86f6e9)
- [S Learner Uplift Model for Individual Treatment Effect and Customer Segmentation in Python](https://medium.com/grabngoinfo/s-learner-uplift-model-for-individual-treatment-effect-and-customer-segmentation-in-python-9d410746e122)
- [How to Use R with Google Colab Notebook](https://medium.com/p/how-to-use-r-with-google-colab-notebook-610c3a2f0eab)

# References
* [StatQuest Support Vector Machines Part 1](https://www.youtube.com/watch?v=efR1C6CvhmE) 
* [Sklearn Support Vector Classification Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
* [RBF SVM parameters](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html)